In [8]:
import pandas as pd
import plotly.graph_objs as go
from datetime import datetime

df = pd.read_excel('CHDS_raw_Data.xlsx', sheet_name="INCIDENT")

# Convert date to datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Group the data by year and count the number of incidents
df_grouped = df.groupby(pd.Grouper(key='Date', freq='Y')).count().reset_index()

# Rename the columns
df_grouped = df_grouped.rename(columns={'Date': 'Year', 'Incident_ID': 'Number of Incidents'})

df = df_grouped

# Create the scatter plot
scatter = go.Scatter(
    x=df['Year'],
    y=df['Number of Incidents'],
    mode='lines+markers',
    line=dict(color='red', width=2),
    marker=dict(size=8, color='red'),
    text=[f"Number of incidents: {y}<br>Year: {x.strftime('%Y')}" for x, y in zip(df['Year'], df['Number of Incidents'])],
    hoverinfo='text',
    hoverlabel=dict(bgcolor='lightgray', font=dict(color='Black'))
)
# Define the animation frames for the line plot
line_frames = [go.Frame(data=[go.Scatter(
    x=df['Year'][:k+1],
    y=df['Number of Incidents'][:k+1],
    mode='lines+markers',
    line=dict(color='red', width=2),
    marker=dict(size=8, color='red')
)]) for k in range(len(df))]

# Create the animation figure
fig = go.Figure(
    data=[scatter],
    layout=go.Layout(
        xaxis=dict(title='Year', range=[datetime.strptime("1969-01-01", '%Y-%m-%d'), datetime.strptime("2024-01-01", '%Y-%m-%d')]),
        yaxis=dict(title='Number of Incidents', range=[df['Number of Incidents'].min()-5, df['Number of Incidents'].max()+5]),
        title='Number of School Shooting Incidents by Year (1970-2022)',
        hovermode='x',
        plot_bgcolor='white'
    ),
    frames=line_frames,
)

# Add the play and pause buttons to the animation figure
updatemenu = [dict(
    type='buttons',
    showactive=True,
    x=0.1,
    y=1,
    xanchor='right',
    yanchor='top',
    buttons=[dict(
        label='Play',
        method='animate',
        args=[None, {'frame': {'duration': 50, 'redraw': True}, 'fromcurrent': True, 'transition': {'duration': 0}}]),
             dict(
        label='Pause',
        method='animate',
        args=[[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate', 'transition': {'duration': 0}}])]
)]

fig.update_layout(updatemenus=updatemenu)

# Show the animation figure
fig.show(renderer="browser")
fig.write_html("incidents_per_year.html")
